In [ ]:
import pandas as pd
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

tabela_gases = pd.read_csv('cityiot.history.csv')

tabela_gases['D'] = pd.to_datetime(tabela_gases['D'])

grafico = go.Figure()

grafico.add_trace(go.Scatter(x=tabela_gases['D'], y=tabela_gases['TVOC'], mode='lines',name='Concentracao TVOC (ppb) ao longo do tempo',\
                             line=dict(color = 'blue', width=3), visible = True))

grafico.add_trace(go.Scatter(x=tabela_gases['D'], y=tabela_gases['G'], mode='lines',name='Concentracao de gases G (ppm) ao longo do tempo',\
                             line=dict(color = 'black', width=3), visible = False))




tvoc_faixas = [dict(y0=0, y1=250, line_width=0.5, fillcolor='green', opacity=0.5),\
               dict(y0=250, y1=2000, line_width=0.5, fillcolor='yellow', opacity=0.5),\
               dict(y0=2000, y1=3000, line_width=0.5, fillcolor='red', opacity=0.5)]

g_faixas = [dict(y0=0, y1=700 , line_width = 0.5, fillcolor='darkgreen', opacity = 0.5),\
            dict(y0= 700, y1=1100,line_width = 0.5, fillcolor='lightgreen', opacity = 0.5),\
            dict(y0= 1100, y1=1600,line_width = 0.5, fillcolor='yellow', opacity = 0.5),\
            dict(y0= 1600, y1=2000,line_width = 0.5, fillcolor='orange', opacity = 0.5),\
            dict(y0= 2000, y1=3000,line_width = 0.5, fillcolor='red', opacity = 0.5)]

for r in tvoc_faixas:
  grafico.add_shape(type='rect', xref = 'paper', x0=0, x1=1, **r)


grafico.update_layout(updatemenus=[dict(buttons =[dict(label = 'TVOC', method = 'update', args =[{'visible' :[True, False]},\
                  {'title' : 'Concentracao TVOC (ppb) ao longo do tempo',\
                   'shapes': [dict(type='rect', xref = 'paper', x0=0, x1=1, **r)for r in tvoc_faixas]}]),\
                  dict(label = 'G', method = 'update', args = [{'visible' : [False,True]},\
                  {'title' : 'Concentracao G (ppm) ao longo do tempo',\
                   'shapes':[dict(type='rect', xref = 'paper', x0=0, x1=1, **r)for r in g_faixas]}] )],\
                                        direction = 'down', showactive = True , x=1.15, y= 1.15,\
                                        xanchor = 'left', yanchor = 'top')],\
                      title = 'Concentracao de gases ao longo do tempo', xaxis_title = 'Data', yaxis_title = 'Concentracao')


grafico.show()





